In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import pickle
import math
import scipy.optimize as optim
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

import covid19_prepare_data as prepare_data

import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING)

In [2]:
def fetch_data():
    prepare_data.build_covid19_data()

In [3]:
# Define funcion with the coefficients to estimate
def func_logistic(t, a, b, c):
    return c / (1 + a * np.exp(-b*t))

In [7]:
def detect_growth():
    countries_processed = 0
    countries_stabilized = 0
    countries_increasing = 0
    
    countries_list = []
    
    df = pd.read_csv('data/covid19_data.csv', parse_dates=True)
    columns = df.columns.values
    for column in columns:
        if column.endswith('_cases'):
            data = pd.DataFrame(df[column].values)
            
            data = data.reset_index(drop=False)
            data.columns = ['Timestep', 'Total Cases']
            
            # Randomly initialize the coefficients
            p0 = np.random.exponential(size=3)

            # Set min bound 0 on all coefficients, and set different max bounds for each coefficient
            bounds = (0, [100000., 1000., 1000000000.])

            # Convert pd.Series to np.Array and use Scipy's curve fit to find the best Nonlinear Least Squares coefficients
            x = np.array(data['Timestep']) + 1
            y = np.array(data['Total Cases'])
            
            try:
                (a,b,c),cov = optim.curve_fit(func_logistic, x, y, bounds=bounds, p0=p0, maxfev=1000000)
                
                # The time step at which the growth is fastest
                t_fastest = np.log(a) / b
                i_fastest = func_logistic(t_fastest, a, b, c)
                
                res_df = df[['Report_Date', column]].copy()
                res_df['fastest_grow_day'] = t_fastest
                res_df['fastest_grow_value'] = i_fastest
                res_df['growth_stabilized'] = t_fastest <= x[-1]
                res_df['timestep'] = x
                res_df['res_func_logistic'] = func_logistic(x, a, b, c)
            
                if t_fastest <= x[-1]:
                    print('Growth stabilized:', column, '| Fastest grow day:', t_fastest, '| Infections:', i_fastest)
                    res_df['cap'] = func_logistic(x[-1] + 10, a, b, c)
                    countries_stabilized += 1
                else:
                    print('Growth increasing:', column, '| Fastest grow day:', t_fastest, '| Infections:', i_fastest)
                    res_df['cap'] = func_logistic(i_fastest + 10, a, b, c)
                    countries_increasing += 1
                
                countries_processed += 1
                countries_list.append(column)
                
                res_df.to_csv('data/covid19_processed_data_' + column + '.csv')
            except RuntimeError:
                print('No fit found for: ', column)
                
    d = {'countries_processed': [countries_processed], 'countries_stabilized': [countries_stabilized], 'countries_increasing': [countries_increasing]}
    df_c = pd.DataFrame(data=d)
    df_c.to_csv('data/covid19_stats_countries.csv')
    
    df_countries = pd.DataFrame(countries_list)
    df_countries.to_csv('data/covid19_countries_list.csv')

# detect_growth()

Growth increasing: Afghanistan_cases | Fastest grow day: 82.42708666625896 | Infections: 626.533932296402
Growth stabilized: Albania_cases | Fastest grow day: 68.70098655373504 | Infections: 225.24667072846725
Growth increasing: Algeria_cases | Fastest grow day: 83.31720379507905 | Infections: 2693.152978795934
Growth stabilized: Andorra_cases | Fastest grow day: 71.05214317374082 | Infections: 394.9685266770038
Growth increasing: Angola_cases | Fastest grow day: 80.69371053720594 | Infections: 12.109076279863695
Growth increasing: Antigua and Barbuda_cases | Fastest grow day: 83.54255190598701 | Infections: 31.88272285430539
Growth increasing: Argentina_cases | Fastest grow day: 78.85899840034882 | Infections: 2149.903513862416
Growth increasing: Armenia_cases | Fastest grow day: 74.79352595850537 | Infections: 827.7130759843484
Growth stabilized: Australia_australian capital territory_cases | Fastest grow day: 70.34683767701372 | Infections: 78.46901460595765
Growth stabilized: Austr

In [9]:
def build_model(country):
    df = pd.read_csv('data/covid19_processed_data_' + country + '.csv', parse_dates=True)
    df_ = df.copy()
    df = df[['Report_Date', country, 'cap']].dropna()
    
    df.columns = ['ds', 'y', 'cap']
    
    m = Prophet(growth="logistic")
    m.fit(df)

    future = m.make_future_dataframe(periods=20)
    future['cap'] = df['cap'].iloc[0]

    forecast = m.predict(future)
    
    res_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds').y).reset_index()
    res_df['current_date'] = df['ds'].iloc[-1]
    res_df['fastest_growth_day'] = df_['fastest_grow_day'].iloc[-1]
    res_df['growth_stabilized'] = df_['growth_stabilized'].iloc[-1]
    res_df['current_day'] = df_['timestep'].iloc[-1]
    res_df['cap'] = df['cap'].iloc[0]
    
    res_df.to_csv('data/covid19_forecast_data_' + country + '.csv')
    
    print('Processed:', country)
    
#     fig1 = m.plot(forecast)
#     fig1.set_size_inches(18.5, 8.5)
#     datenow = datetime(2020, 4, 1)
#     dateend = datenow + timedelta(days=20)
#     datestart = dateend - timedelta(days=71)
#     plt.xlim([datestart, dateend])
#     plt.title("COVID19 forecast: " + country, fontsize=20)
#     plt.xlabel("Day", fontsize=20)
#     plt.ylabel("Infections", fontsize=20)
#     plt.axvline(datenow, color="k", linestyle=":")
#     plt.show()
    
#     print(res_df[['ds', 'y', 'yhat', 'yhat_lower', 'yhat_upper', 'current_date', 'fastest_growth_day', 'growth_stabilized', 'current_day']].tail(30))
    
# build_model('Lithuania_cases')

In [10]:
def calculate_forecast():
    df = pd.read_csv('data/covid19_data.csv', parse_dates=True)
    columns = df.columns.values
    for column in columns:
        if column.endswith('_cases'):
            build_model(column)
    print('Forecast calculation completed')
    
# calculate_forecast()

/opt/conda/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



Processed: Afghanistan_cases
Processed: Albania_cases
Processed: Algeria_cases
Processed: Andorra_cases
Processed: Angola_cases
Processed: Antigua and Barbuda_cases
Processed: Argentina_cases
Processed: Armenia_cases
Processed: Australia_australian capital territory_cases
Processed: Australia_new south wales_cases
Processed: Australia_northern territory_cases
Processed: Australia_queensland_cases
Processed: Australia_south australia_cases
Processed: Australia_tasmania_cases
Processed: Australia_victoria_cases
Processed: Australia_western australia_cases
Processed: Austria_cases
Processed: Azerbaijan_cases
Processed: Bahamas_cases
Processed: Bahrain_cases
Processed: Bangladesh_cases
Processed: Barbados_cases
Processed: Belarus_cases
Processed: Belgium_cases
Processed: Benin_cases
Processed: Bhutan_cases
Processed: Bolivia_cases
Processed: Bosnia_cases
Processed: Brazil_cases
Processed: Brunei_cases
Processed: Bulgaria_cases
Processed: Burkina Faso_cases
Processed: Cabo Verde_cases
Proce